In [1]:
import pandas as pd
import os
import json

os.getcwd()

'/Users/danaru'

In [3]:
file_name = "subsampled_drug_info.json"
# Open and read the JSON file
with open(file_name, 'r') as file:
    data = json.load(file)

df = pd.DataFrame(data)

In [20]:
drugs = df.columns
rows = df.index
disease_prompts = dict()


for drug in drugs:
    diseases = df[drug]['Disease Information']
    # print(diseases)
    receptors = df[drug]['Additional Information']['targets']
    rec_data = dict()
    k = list(df[drug]['Active Ingredients'].keys())
    smile = df[drug]['Active Ingredients'][k[0]]
    #collect all receptors per drug
    for r in receptors:
        rec_data[r['Label']] = {'Amino acid sequence' : r['Detailed Info']['Amino acid sequence'], 
                                'General Function': r['Detailed Info']['General Function'],
                                'Specific Function' : r['Detailed Info']['Specific Function']}                   
    for dis in diseases:
        if diseases[dis]==None:
            info =''
        else:
            info = ', is '+ diseases[dis]
        other_diseases = [disease for disease in diseases if disease != dis]
        # Choose the first 5 diseases from the list of other diseases
        selected_diseases = other_diseases[:5]
        disease_prompts[dis] = {'info': info, 'receptors': rec_data, 'drug_smile' : smile, 'similar_conditions': selected_diseases}
        

In [55]:
def num_tokens_from_string(string: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")
    num_tokens = len(encoding.encode(string))
    return num_tokens

In [31]:
templates_prompts_w_smiles = dict()
for d in disease_prompts:
    receptors = []
    started =False
    #going over the receptors per disease:
    for r in disease_prompts[d]['receptors']:
        started = True
        gf=disease_prompts[d]['receptors'][r]['General Function']
        sf = disease_prompts[d]['receptors'][r]['Specific Function']
        ac = disease_prompts[d]['receptors'][r]['Amino acid sequence']
        ac = '\n'.join(ac.split('\n')[1:]) #remove first line with not sequence stuff
        if started:
            receptors.append('. \n')
        receptor = f'Receptor {r}'
        receptors.append(receptor)
        if gf is not None and gf != '':
            str = f'Has general function: {gf},'
            receptors.append(str)
        if sf is not None and sf != '':
            str = f'Has specific function function: {sf},'
            receptors.append(str)
        if ac is not None and ac != '':
            str = f'Its amino acid sequence is: {ac}.'
            receptors.append(str)
    allreceptors = ' '.join(receptors)
    i=disease_prompts[d]['info']
    
    similar_conditions = disease_prompts[d]['similar_conditions']
    if similar_conditions != None and similar_conditions != '' and similar_conditions != []:
        similar_conditions = ', '.join(similar_conditions)
        similar_disease = f'Some associated conditions are {similar_conditions}.'
        
    prompt = f'You are an expert in drug discovery, please generate a novel binder molecule for the following disease. Respond with a SMILE string of a hypothetical possible binder molecule. The disease is {d} {i}. {similar_disease} It has the following receptors: {allreceptors}'
    smile = disease_prompts[d]['drug_smile']
    num_tokens = num_tokens_from_string(prompt+smile)
    while num_tokens>3950:
        last_receptor_index = prompt.rfind("Receptor")
        prompt = prompt[:last_receptor_index]
        num_tokens = num_tokens_from_string(prompt+smile)

    templates_prompts_w_smiles[d] = {'prompt':prompt, 'smile_output': smile}
            

In [60]:
#Example
a = templates_prompts_w_smiles['Muscle Spasms']['prompt']


In [36]:
import tiktoken

In [37]:
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

/Users/danaru/Library/Python/3.8/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [38]:
def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")
    num_tokens = len(encoding.encode(string))
    return num_tokens

In [53]:
for i in templates_prompts_w_smiles.keys():
    a = templates_prompts_w_smiles[i]['prompt']
    num = num_tokens_from_string(a, "cl100k_base")
    if num>3000:
        
        print(num)

3065
3162
3109
3039
3179
3018
3101
3036
3117
3170
3232
3235
3124
3227
3119
3348
3267
3188
3120
3120
3299


In [45]:
templates_prompts_w_smiles.keys()

dict_keys(['Anginal Pain', 'Cardiovascular Complications', 'Cardiovascular Disease', 'Coronary Artery Disease', 'Coronary artery thrombosis', 'Dysbetalipoproteinemia', 'Fredrickson Type III lipidemia', 'Heterozygous Familial Hypercholesterolemia', 'High Cholesterol', 'Homozygous Familial Hypercholesterolaemia', 'Hospitalizations', 'Hypertension, Essential Hypertension', 'Hypertriglyceridemias', 'Mixed Dyslipidemias', 'Mixed Hyperlipidemia', 'Myocardial Infarction', 'Non-familial hypercholesterolemia', 'Nonfatal Myocardial Infarction', 'Postoperative Thromboembolism', 'Primary Hypercholesterolemia', 'Stroke', 'Thrombosis', 'Transient Ischemic Attack', 'Elevation of serum triglyceride levels', 'Heterozygous familial hyperlipidemia', 'Non-familial hyperlipidemia', 'Primary Hyperlipidemia', 'Revascularization procedures', 'Chronic Pain', 'Severe Pain', 'Actinomycosis', 'Acute Rheumatic Fever', 'Anthrax', 'Bloodstream Infections', 'Clostridium Botulinum Toxin Adverse Reaction', 'Diphtheria'

In [50]:
num_tokens_from_string(templates_prompts_w_smiles['Headache']['prompt'], "cl100k_base")

2249